<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/P3-Practica-Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭕ Práctica de clasificación

## 1. Práctica en clase



Vamos a trabajar con el dataset PIMA. El objetivo es

 *Can you build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?*

Número de embarazos (Pregnancies):

    Cantidad total de embarazos que la paciente ha tenido.

Concentración de glucosa en plasma (Glucose):

    Nivel de glucosa en sangre a 2 horas en una prueba de tolerancia oral a la glucosa (mg/dL).

    Valores normales: 70–99 mg/dL (en ayunas).

Presión arterial diastólica (BloodPressure):

    Presión sanguínea en mm Hg.

    Valor normal: ≈80 mm Hg.

Grosor del pliegue cutáneo (SkinThickness):

    Medida (en mm) del pliegue cutáneo del tríceps para estimar grasa corporal.

    Valor normal: 10–40 mm.

Insulina sérica (Insulin):

    Nivel de insulina en sangre a 2 horas (muU/mL).

    Valor normal en ayunas: 2–25 μU/mL.

Índice de masa corporal (IMC) (BMI):

    Peso (kg) / altura² (m²).

    Rango saludable: 18.5–24.9.

Función pedigrí de diabetes (DiabetesPedigreeFunction):

    Puntuación que estima la influencia genética basada en antecedentes familiares de diabetes.

    Valor alto: Mayor riesgo genético.

Edad (Age):

    Edad de la paciente en años.

Resultado (Outcome):

    0: No diabetes.

    1: Diabetes diagnosticada.

Vamos a construir dos modelos de clasificación y compararlos

¿Qué clasificador tiene mejor rendimiento en este dataset? Para esto **toma en cuenta el recall en el conjunto de prueba**.

Leamos el dataset

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/diabetes.csv'
df = pd.read_csv(url)
df

Usa el método describe del dataframe para observar los rangos de los datos

Los ceros en varias columnas representan valores faltantes,

🔵 ¿En cuáles columnas no tienen sentido los valores 0?

Observa los ceros en algunas de estas columnas donde no tienen sentido los ceros

In [ ]:
df[df['Insulin']==0].shape

🔴 **OPCIONAL**: Haz imputación de los datos con valores 0 en las columnas que consideres necesario

In [ ]:
from sklearn.impute import SimpleImputer


🔴 Divide el dataset en train/test usando 80% para entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split



🔴 Re escala los datos usando MinMaxScaler o StandardScaler. Recuerda el **data leakage**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler



🔴 Entrena un modelo de SVM y evalua usando la métrica recall en el conjunto de prueba, comparala con el recall en el conjunto de entrenamiento. ¿Hay señales de overfitting?

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, recall_score

🔴 Entrena un modelo de Decision Tree / Random Forest y evalua usando la métrica recall en el conjunto de prueba, comparala con el recall en el conjunto de entrenamiento. ¿Hay señales de overfitting?

En este modelo, ¿cuáles son las variables más importantes para predecir la diabetes? ¿Tiene sentido la conclusión?

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score



🔴 🔴 Realiza un grid search usando los parámetros descritos a continuación:

Modelo 1:
* SVM
 * C: 0.1,1,10,100
 * kernel: lineal, polinomial, rbf
 * grados (polinomial): 2,3,5

Modelo 2. Escoge uno de los siguientes:
* Decision Tree
 * criterion: gini, entropy, log_loss
 * max_depth: None, 10, 20,
 * min_samples_split: 2, 3, 5
* Random Forest
 * criterion: gini, entropy, log_loss
 * max_depth: None, 10, 20,
 * min_samples_split: 2, 3, 5
* Regresión Logística
 * C: 0.1,1,10
 * penalty: l1, l2, elasticnet, None

In [ ]:
from sklearn.model_selection import GridSearchCV
